In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')


In [2]:
# load the provided data
# train_features = pd.read_csv('./data/train_features.csv')
# train_labels = pd.read_csv('./data/train_labels.csv')
# test_features = pd.read_csv('./data/test_features.csv')

In [4]:
# get the 'output' column from labels
#train_outputs = train_labels['total_cases']

train_data = pd.read_csv('./data2/TrainingData.csv') #train_features
test_data = pd.read_csv('./data2/TestDataWithHeaders.csv') #test_features

#train_data['total_cases'] = train_outputs

#y_train = train_data["total_cases"].values

train_data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A27,A28,A29,A30,A31,A32,A33,A34,A35,Class
0,0.36,0.45,0.47,0.37,0.33,0.40,0.42,0.31,0.33,0.40,...,0.53,0.41,0.35,0.47,0.50,0.39,0.33,0.42,0.44,C
1,0.33,0.40,0.42,0.31,0.33,0.40,0.40,0.33,0.31,0.40,...,0.50,0.39,0.33,0.42,0.44,0.34,0.33,0.39,0.41,C
2,0.33,0.40,0.40,0.33,0.31,0.40,0.40,0.31,0.33,0.37,...,0.44,0.34,0.33,0.39,0.41,0.31,0.33,0.39,0.41,C
3,0.31,0.40,0.40,0.31,0.33,0.37,0.40,0.31,0.31,0.37,...,0.41,0.31,0.33,0.39,0.41,0.31,0.33,0.40,0.41,C
4,0.33,0.37,0.40,0.31,0.31,0.37,0.38,0.30,0.31,0.40,...,0.41,0.31,0.33,0.40,0.41,0.31,0.31,0.42,0.43,C


In [5]:
# check the decoration and select best effecting and not effecting coloms
train_data.info()
print('_'*40)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 36 columns):
A1       5000 non-null float64
A2       5000 non-null float64
A3       5000 non-null float64
A4       5000 non-null float64
A5       5000 non-null float64
A6       5000 non-null float64
A7       5000 non-null float64
A8       5000 non-null float64
A9       5000 non-null float64
A10      5000 non-null float64
A11      5000 non-null float64
A12      5000 non-null float64
A13      5000 non-null float64
A14      5000 non-null float64
A15      5000 non-null float64
A16      5000 non-null float64
A17      5000 non-null float64
A18      5000 non-null float64
A19      5000 non-null float64
A20      5000 non-null float64
A21      5000 non-null float64
A22      5000 non-null float64
A23      5000 non-null float64
A24      5000 non-null float64
A25      5000 non-null float64
A26      5000 non-null float64
A27      5000 non-null float64
A28      5000 non-null float64
A29      5000 non-null fl

In [7]:
# analyze output
# descriptive statistics summary
train_data['Class'].describe()
#histogram
sns.distplot(train_data['Class']);

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [9]:
#correlation matrix
corrmat = train_data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);
plt.show()

In [8]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'total_cases')['total_cases'].index
cm = np.corrcoef(train_data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [10]:
# remove unwanted coloms according to heatmap and our knowladge

train_data.drop('year', axis=1, inplace=True)
train_data.drop('ndvi_ne', axis=1, inplace=True)

test_data.drop('year', axis=1, inplace=True)
test_data.drop('ndvi_ne', axis=1, inplace=True)

In [11]:
#check and preprocess missing values in train data
total = train_data.isnull().sum().sort_values(ascending=False)
percent = (train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
ndvi_nw,52,0.035714
station_diur_temp_rng_c,43,0.029533
station_avg_temp_c,43,0.029533
ndvi_se,22,0.015110
ndvi_sw,22,0.015110
station_precip_mm,22,0.015110
station_max_temp_c,20,0.013736
station_min_temp_c,14,0.009615
reanalysis_sat_precip_amt_mm,13,0.008929
precipitation_amt_mm,13,0.008929


In [12]:
# remove coloms which have missing values than 15% for train data
train_data.drop((missing_data[missing_data['Percent'] > 0.15]).index,1)

# remove rows which have missing values for spesific coloms, eg: total_cases colom for train data
#train_data.drop(train_data.loc[train_data['total_cases'].isnull()].index)

# fill missing values for train data
train_data["station_min_temp_c"] = train_data["station_min_temp_c"].fillna("None")  # train_data['station_min_temp_c'].mode()[0]


In [13]:
#check and preprocess missing values in test data
total = test_data.isnull().sum().sort_values(ascending=False)
percent = (test_data.isnull().sum()/test_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
station_diur_temp_rng_c,12,0.028846
station_avg_temp_c,12,0.028846
ndvi_nw,11,0.026442
station_min_temp_c,9,0.021635
station_precip_mm,5,0.012019
station_max_temp_c,3,0.007212
precipitation_amt_mm,2,0.004808
reanalysis_air_temp_k,2,0.004808
reanalysis_avg_temp_k,2,0.004808
reanalysis_dew_point_temp_k,2,0.004808


In [14]:

# remove coloms which have missing values than 15% for test data
train_data.drop((missing_data[missing_data['Percent'] > 0.15]).index,1)

# fill missing values for test data
train_data["station_min_temp_c"] = train_data["station_min_temp_c"].fillna("None")  # train_data['station_min_temp_c'].mode()[0]


In [19]:
# remove output because don't want to train the models
y_train = train_data["Class"].values
cols_to_transform = [ 'F', 'E', 'D', 'C', 'B', 'A' ]
y_train = y_train.get_dummies( columns = cols_to_transform )
train_data.drop(['Class'], axis=1, inplace=True)

KeyError: 'Class'

In [9]:
#Validation function

n_folds = 5


def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_data.values)
    rmse= np.sqrt(-cross_val_score(model, train_data.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [11]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

# score = rmsle_cv(lasso)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

# score = rmsle_cv(ENet)
# print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [18]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

KRR.fit(train_data,y_train)
prediction5=KRR.predict(test_data)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction5,y_train))

# score = rmsle_cv(KRR)
# print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ValueError: could not convert string to float: 'E'

In [22]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
# score = rmsle_cv(GBoost)
# print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [22]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

# score = rmsle_cv(model_xgb)
# print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)


# score = rmsle_cv(model_lgb)
# print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [15]:
RandomForest_model = RandomForestRegressor(n_estimators=20, max_features='auto',
                                 max_depth=6, min_samples_leaf=0.005,
                                 criterion='mae', min_weight_fraction_leaf=0.1
                                , warm_start=True)
# score = rmsle_cv(RandomForest_model)
# print("RandomForest_model score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [17]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)



In [ ]:
# Simplest Stacking approach : Averaging base models
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models

    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self

    # Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [ ]:
averaged_models = AveragingModels(models = (clf, RandomForest_model, KRR, lasso))

model=KNeighborsClassifier() 
model.fit(train_X,train_Y)
prediction5=model.predict(test_X)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction5,test_Y))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# Stacking averaged Models Class
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)

        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred

        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    # Do the predictions of all base models on the test data and use the averaged predictions as
    # meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_])
        return self.meta_model_.predict(meta_features)


In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
stacked_averaged_models.fit(train_data.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train_data.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(train_data.values))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
model_xgb.fit(train_data, y_train)
xgb_train_pred = model_xgb.predict(train_data)
xgb_pred = np.expm1(model_xgb.predict(train_data))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
model_lgb.fit(train_data, y_train)
lgb_train_pred = model_lgb.predict(train_data)
lgb_pred = np.expm1(model_lgb.predict(train_data.values))
print(rmsle(y_train, lgb_train_pred))


In [ ]:
print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

In [ ]:
# ensemble prediction
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [25]:
# save results
sub = pd.DataFrame()
sub['city'] = train_labels['city']
sub['year'] = train_labels['year']
sub['weekofyear'] = train_labels['weekofyear']
sub['total_cases'] = ensemble
sub.to_csv('submission.csv',index=False)

NameError: name 'ensemble' is not defined